In [23]:
# 利用 Logistic Regression 模型來區分資料成兩類



# Logistic Regression 特性 : 

# 1. 本身具有機率的成分，不光是預測是哪個類別，而是預測樣本屬於該類別的 " 機率 "

# 2. 雖然轉換成機率的 Sigmoid 函數是非線性，但本身模型還是 ax + b 線性，所以算線性模型，比較適合線性資料

# 3. 是 ANN 類神經網路裡面一個神經元 , 函數是 Sigmoid 機率函數的特例



# a. 下面範例為簡單的四個點，分別在 y 軸的上面和下面，基本上可以用線性的方法來回歸區分出兩類

# 1. 確實線性回歸可以做到基本的分類，和 Logistic Regression 一樣

# 2. 但是會缺乏機率的結果呈現和解釋，沒有使用 Sigmoid 函數將結果限制在 0 到 1 之間


from sklearn.linear_model import LinearRegression
from sklearn.metrics import classification_report
import numpy

x = numpy.array([[2.5,1.5],[-1.7,2.7],[-1.8,-0.9],[1.6,-1.3]])
y = numpy.array([1,1,0,0])

model = LinearRegression()
model.fit(x,y)

y_prediction = model.predict(x)

print(y_prediction)

y_map = []

# 要自己做閥值和區分

for item in y_prediction : 

    if item >= 0.5 : 

        y_map.append(1)

    else : 

        y_map.append(0)

report = classification_report(y,y_map,labels=["0","1"])

print(report)



# b. 可以發現分類效果還蠻好的 ! 但是 

# 1. y_prediction 會超過 0 ~ 1 之間，不像是機率的感覺

# 2. 所以要有 Sigmoid 函數的轉換來得到機率，下面做一個用 Sigmoid 函數轉換 y_prediction 的版本



x = numpy.array([[2.5,1.5],[-1.7,2.7],[-1.8,-0.9],[1.6,-1.3]])
y = numpy.array([1,1,0,0])

model = LinearRegression()
model.fit(x,y)

print(model.coef_,model.intercept_)


# 這邊做 Sigmoid 函數，然後要做一個函數來處理 sigmoid 函數的代入值為陣列時也傳回陣列的版本

# 基本上這邊因為用 numpy.exp() 所以可以處理陣列，但是如果是 math.exp() 就一定要多做一個函數來處理

def sigmoid(u) :  
    
    value = 1 / (1 + numpy.exp(-u))

    return value

linear_y = model.predict(x)

# 使用 frompyfunc 來將函數轉成可以回傳陣列

# 後面兩個參數是每次輸入 1 個值 ( u_value ) 就產生 1 個值 ( sigmoid(u) )，最後形成陣列

array_sigmoid = numpy.frompyfunc(sigmoid,1,1)

# 使用上有 lamda 的成分，上面 frompyfunc 裡面的 sigmoid 不用放參數，放在下面呼叫時

y_prediction = array_sigmoid(linear_y)

print(y_prediction)

y_map_2 = []

for item in y_prediction : 

    if item >= 0.5 : 

        y_map_2.append(1)

    else : 

        y_map_2.append(0)

report_2 = classification_report(y,y_map_2,labels=["0","1"])

print(report_2)



# c. 可以發現分類的 y_prediction 確實變成機率

# 1. 但是分類的機率會變成有點集中，不太好區分

# 2. 因為 Logistic Regression 的機率分布假設和 Linear Regression 不同，因此 Linear Regression 預測值套上 Sigmoid 產生機率會怪怪的

# 3. 也因為機率分布假設不同，因此 Logistic Regression 的閥值不會是線性一半一半切分，而是有 log 的非線性特性

# 4. Linear Regression 是用 MSE 來求參數，而 Logistic Regression 是用 MLE 來求參數，MLE 才有機率的概念

# 5. 下面修正成 Logistic Regression 並且用 Sigmoid 函數看預測值，就可以發現機率就明顯比較分散



from sklearn.linear_model import LogisticRegression

x = numpy.array([[2.5,1.5],[-1.7,2.7],[-1.8,-0.9],[1.6,-1.3]])
y = numpy.array([1,1,0,0])

model = LogisticRegression()
model.fit(x,y)

print(model.coef_,model.intercept_)


# 這邊把 y_prediction 改成自己算，看看中間 Logistic Regression 產生的值和 Sigmoid 函數轉換得到的機率

# ( Logistic Regression 的 model.predict 是預設 0.5 以上就視為 1 ，小於就視為 0，這邊一樣比照 )

y_logistic = x[:,0]*model.coef_[0,0] + x[:,1]*model.coef_[0,1] + model.intercept_

print(y_logistic)

# 也可以用 scipy.special 的 expit 來直接求 sigmoid 函數值

from scipy.special import expit

y_sigmoid = expit(y_logistic)

print(y_sigmoid)


y_map_3 = []

for item in y_sigmoid : 

    if item >= 0.5 : 

        y_map_3.append(1)

    else : 

        y_map_3.append(0)

report_3 = classification_report(y,y_map_3,labels=["0","1"])

print(report_3)


[ 0.95434492  1.03673227 -0.04773707  0.05665989]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         2

   micro avg       1.00      1.00      1.00         4
   macro avg       1.00      1.00      1.00         4
weighted avg       1.00      1.00      1.00         4

[0.06592971 0.2994101 ] 0.34040549248996177
[np.float64(0.7219881357765339) np.float64(0.7382190026496331)
 np.float64(0.4880679972525585) np.float64(0.5141611846642938)]
              precision    recall  f1-score   support

           0       1.00      0.50      0.67         2
           1       0.67      1.00      0.80         2

   micro avg       0.75      0.75      0.75         4
   macro avg       0.83      0.75      0.73         4
weighted avg       0.83      0.75      0.73         4

[[0.20039829 1.00704966]] [-0.53125836]
[ 1.48031185  1.84709861 -1.79831998 -1.51978565]
[0.81461968 0.86378609 0.1420557  0